In [22]:
import numpy as np # linear algebra
import pandas as pd
import os
for dirname, _, filenames in os.walk('./input/heart.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [25]:
#Load data into spark 
data = spark.read.csv('./input/heart.csv', header = True)

In [26]:
df = pd.read_csv('./input/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [28]:
#Columns in data
data.printSchema()

root
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- cp: string (nullable = true)
 |-- trestbps: string (nullable = true)
 |-- chol: string (nullable = true)
 |-- fbs: string (nullable = true)
 |-- restecg: string (nullable = true)
 |-- thalach: string (nullable = true)
 |-- exang: string (nullable = true)
 |-- oldpeak: string (nullable = true)
 |-- slope: string (nullable = true)
 |-- ca: string (nullable = true)
 |-- thal: string (nullable = true)
 |-- target: string (nullable = true)



In [29]:
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
  for name in names: 
     df = df.withColumn(name, df[name].cast(newType))
  return df 
int_columns = ['age','sex','cp', 'trestbps','chol','fbs','restecg','thalach','exang','slope','ca','thal','target']
float_column = ['oldpeak']
data = convertColumn(data, int_columns, IntegerType())
data = convertColumn(data, float_column, FloatType())

In [30]:
data.persist()

DataFrame[age: int, sex: int, cp: int, trestbps: int, chol: int, fbs: int, restecg: int, thalach: int, exang: int, oldpeak: float, slope: int, ca: int, thal: int, target: int]

In [31]:
#Take one sample
data.take(1)

[Row(age=52, sex=1, cp=0, trestbps=125, chol=212, fbs=0, restecg=1, thalach=168, exang=0, oldpeak=1.0, slope=2, ca=2, thal=3, target=0)]

In [32]:
data.describe().show() # It is hard to see

+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|summary|              age|               sex|                cp|          trestbps|             chol|                fbs|           restecg|           thalach|              exang|           oldpeak|             slope|                ca|              thal|            target|
+-------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+
|  count|             1025|              1025|              1025|              1025|             1025|               1025|              1025|              1025|            

In [33]:
data.describe('target').show()

+-------+------------------+
|summary|            target|
+-------+------------------+
|  count|              1025|
|   mean|0.5131707317073171|
| stddev|0.5000704980788011|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [34]:
data.describe('age').show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|             1025|
|   mean|54.43414634146342|
| stddev|9.072290233244278|
|    min|               29|
|    max|               77|
+-------+-----------------+



In [35]:
#Count number of samples
data.count()

1025

In [36]:
#Change spark to pandas dataframe
pd_data =data.toPandas()
pd_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [37]:
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int32  
 1   sex       1025 non-null   int32  
 2   cp        1025 non-null   int32  
 3   trestbps  1025 non-null   int32  
 4   chol      1025 non-null   int32  
 5   fbs       1025 non-null   int32  
 6   restecg   1025 non-null   int32  
 7   thalach   1025 non-null   int32  
 8   exang     1025 non-null   int32  
 9   oldpeak   1025 non-null   float32
 10  slope     1025 non-null   int32  
 11  ca        1025 non-null   int32  
 12  thal      1025 non-null   int32  
 13  target    1025 non-null   int32  
dtypes: float32(1), int32(13)
memory usage: 56.2 KB


In [38]:
#One-hot encoding maps a categorical feature, represented as a label index, to a binary vector with at most a single one-value indicating the presence of a specific feature value from among the set of all feature values.
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=['sex','cp','fbs','restecg','exang','slope','ca','thal'],
                           outputCols =['sexVec','cpVec','fbsVec','restecgVec','exangVec','slopeVec','caVec','thalVec'])
encoded =  encoder.fit(data).transform(data)
data_encoded = encoded.drop('sex','cp','fbs','restecg','exang','slope','ca','thal')
data_encoded.show()

+---+--------+----+-------+-------+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|age|trestbps|chol|thalach|oldpeak|target|       sexVec|        cpVec|       fbsVec|   restecgVec|     exangVec|     slopeVec|        caVec|      thalVec|
+---+--------+----+-------+-------+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| 52|     125| 212|    168|    1.0|     0|    (1,[],[])|(3,[0],[1.0])|(1,[0],[1.0])|(2,[1],[1.0])|(1,[0],[1.0])|    (2,[],[])|(4,[2],[1.0])|    (3,[],[])|
| 53|     140| 203|    155|    3.1|     0|    (1,[],[])|(3,[0],[1.0])|    (1,[],[])|(2,[0],[1.0])|    (1,[],[])|(2,[0],[1.0])|(4,[0],[1.0])|    (3,[],[])|
| 70|     145| 174|    125|    2.6|     0|    (1,[],[])|(3,[0],[1.0])|(1,[0],[1.0])|(2,[1],[1.0])|    (1,[],[])|(2,[0],[1.0])|(4,[0],[1.0])|    (3,[],[])|
| 61|     148| 203|    161|    0.0|     0|    (1,[],[])|(3,[0],[1.0])|

In [39]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols =['age','trestbps','chol','thalach','oldpeak','sexVec','cpVec','fbsVec','restecgVec','exangVec','slopeVec','caVec','thalVec'],
                           outputCol="features")
assembled= assembler.transform(data_encoded)

data_asb =assembled.select("features","target")
data_asb.show()

+--------------------+------+
|            features|target|
+--------------------+------+
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,6,9,...|     0|
|(22,[0,1,2,3,4,5,...|     0|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,4,5,...|     0|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,4,6,...|     0|
|(22,[0,1,2,3,4,5,...|     1|
|(22,[0,1,2,3,8,10...|     1|
+--------------------+------+
only showing top 20 rows



In [40]:
from pyspark.ml.feature import PCA

pca = PCA(k=13, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(data_asb).transform(data_asb)

data_pca = model.select("pcaFeatures","target")
data_pca.head()

Row(pcaFeatures=DenseVector([-217.5238, 150.2683, 136.8733, 53.3965, -2.6037, 0.9817, -2.2323, 0.768, -0.1088, -1.1132, 0.2432, -0.7249, -1.5901]), target=0)

In [41]:
train_data, test_data = data_pca.randomSplit([.7,.3],seed=1234)

In [42]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize the DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="target", featuresCol="pcaFeatures")

# Fit the data to the model
dt_model = dt.fit(train_data)

# Make predictions on the test data
dt_predictions = dt_model.transform(test_data)

# Calculate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='target', predictionCol='prediction', metricName='accuracy')
dt_accuracy = evaluator.evaluate(dt_predictions)

# Print the test accuracy
print('Test Accuracy = ', dt_accuracy)


Test Accuracy =  0.8841059602649006


In [43]:
from pyspark.ml.classification import RandomForestClassifier
# Initialize
rf = RandomForestClassifier(labelCol="target", featuresCol="pcaFeatures")
# Fit the data to the model
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)
# Calculate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='target',predictionCol='prediction', metricName='accuracy')
rf_accuracy = evaluator.evaluate(rf_predictions)
print('Test Accuracy = ', rf_accuracy)

Test Accuracy =  0.9536423841059603
